<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9D%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Нейронные сети
####Вариант 5:
Просчитать одну итерацию цикла обучения по Δ-правилу однослойной бинарной
неоднородной нейронной сети, состоящей из 2 нейронов и имеющей функции активации: гиперболический тангенс ($k=2$) и пороговую функцию ($Т=0.5$). В качестве обучающей выборки использовать таблицу истинности для операций эквивалентности и конъюнкции (не использовать первую строчку таблицы). Синаптические веса задать случайным образом.

In [60]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it

In [61]:
#Коэффициент обучения (точность) [0;1]
learningRate = 0.6
#Количество слоёв
layer = 1
#количество нейронов на каждом слое
neurons = 2

#--------------------------------------------------------------------

#функции активации
#гиперболический тангенс:
def hyperbolicTangentFunction(S, k = 2):
  return math.tanh(S / k)
#сигмоидная функция:
def sigmoidFunction(S, k = 2):
  return 1.0 / (1 + math.exp(- S * k))
#Пороговая функция:
def thresholdFunction(S, T = 0.5):
  return 1 if S >= T else 0

#--------------------------------------------------------------------

def randomVector(size = 2):
  return [random.uniform(0, 1) for i in range(size)]

def randomNumber():
  return random.uniform(0, 1)

def equiv(a,b): # эквивалентность ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

In [62]:
def deltaRule(data, level = 1, N = 2, nabla = 0.1):
  wi = [[randomNumber() for i in range(N)] for j in range(N)]         #синаптические веса
  si = [0] * N
  yi = [0] * N
  epsi = [0] * N

  level_i = 0
  k = 0
  while(level_i < level): #цикл по слоям
    while(k < len(data)): #цикл по строкам таблицы с данными
      eps = 0
      print("Матрица весов:")
      print(pd.DataFrame(wi))
      print("\nДля x = ( " + str(data[data.columns[0]][k]) + ", " + str(data[data.columns[1]][k]) + ' )')

      for i in range(N):
        for j in range(N):
          si[i] += data[data.columns[j]][k] * wi[j][i]

        yi[i] = hyperbolicTangentFunction(si[i])
        epsi[i] = data[data.columns[N + i]][k] - yi[i]
        eps += epsi[i]**2

      rez_i = pd.DataFrame([si, yi, epsi], index=['S', 'Y расч.', 'EPS'], columns=['neuron 1', 'neuron 2'])
      print(rez_i)
      print("\nСКО: " + str(eps) + "\n\n")

      #вычисление весов для следующей итерации:
      for i in range(N):
        for j in range(N):
          wi[j][i] = wi[j][i] - nabla * epsi[i] * data[data.columns[j]][k]

      k += 1
    level_i += 1


  #return 0

In [64]:
functions = [hyperbolicTangentFunction, thresholdFunction]

table = pd.DataFrame(it.product([0,1], repeat=2), columns=['X1', 'X2'])
table['Y1 = equivalence (~)'] = [equiv(table[table.columns[0]][i], table[table.columns[1]][i]) for i in range(2**neurons)]
table['Y2 = conjunction (^)'] = [(table[table.columns[0]][i] and table[table.columns[1]][i]) for i in range(2**neurons)]

deltaRule(table, layer, neurons, learningRate)

Матрица весов:
          0         1
0  0.061770  0.000268
1  0.736194  0.617260

Для x = ( 0, 0 )
     neuron 1  neuron 2
S         0.0       0.0
Y         0.0       0.0
EPS       1.0       0.0

СКО: 1.0


Матрица весов:
          0         1
0  0.061770  0.000268
1  0.736194  0.617260

Для x = ( 0, 1 )
     neuron 1  neuron 2
S    0.736194   0.61726
Y    0.352326   0.29919
EPS -0.352326  -0.29919

СКО: 0.21364850062629676


Матрица весов:
         0         1
0  0.06177  0.000268
1  0.94759  0.796774

Для x = ( 1, 0 )
     neuron 1  neuron 2
S    0.797964  0.617527
Y    0.379078  0.299312
EPS -0.379078 -0.299312

СКО: 0.23328752778092576


Матрица весов:
          0         1
0  0.289216  0.179855
1  0.947590  0.796774

Для x = ( 1, 1 )
     neuron 1  neuron 2
S    2.034770  1.594156
Y    0.768799  0.662400
EPS  0.231201  0.337600

СКО: 0.1674273974330554




In [ ]:
#1-Количество входов и выходов могут быть различны
#2-Реализовать расчёт промежуточных слоёв